In [8]:
import boto3
import numpy as np
import os
import sagemaker
import json

In [2]:
region = sagemaker.Session().boto_region_name
boto_session = boto3.Session(region_name=region)
sagemaker_boto_client = boto_session.client("sagemaker")

In [3]:
mpg_name = "fraud-detection"

In [18]:
sagemaker_iam_role = sagemaker.get_execution_role()

In [19]:
sagemaker_iam_role

'arn:aws:iam::470086202700:role/sagemaker-developers'

In [129]:
sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)["ModelPackageSummaryList"]

[{'ModelPackageGroupName': 'fraud-detection',
  'ModelPackageVersion': 1,
  'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:470086202700:model-package/fraud-detection/1',
  'ModelPackageDescription': 'XGBoost classifier to detect credit fraud.',
  'CreationTime': datetime.datetime(2024, 1, 9, 19, 10, 41, 620000, tzinfo=tzlocal()),
  'ModelPackageStatus': 'Completed',
  'ModelApprovalStatus': 'Approved'}]

In [5]:
model_package_group_name = 'fraud-detection'

# Create EventBridge client

In [6]:
eventbridge_client = boto3.client('events')

rule_name = 'model-approval-rule-specific-group'
event_pattern = {
    "source": ["aws.sagemaker"],
    "detail-type": ["SageMaker Model Package State Change"],
    "detail": {
        "ModelPackageGroupName": [model_package_group_name],
        "ModelPackageStatus": ["Approved"]
    }
}

In [1]:
# Create update the rule
response = eventbridge_client.put_rule(
    Name=rule_name,
    EventPattern=json.dumps(event_pattern),
    State='ENABLED'
)

NameError: name 'eventbridge_client' is not defined

In [121]:
response

{'RuleArn': 'arn:aws:events:us-east-1:470086202700:rule/model-approval-rule-specific-group',
 'ResponseMetadata': {'RequestId': 'b67343f5-a8b9-4df0-b123-bd6ede8feb5d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b67343f5-a8b9-4df0-b123-bd6ede8feb5d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Wed, 10 Jan 2024 19:37:19 GMT'},
  'RetryAttempts': 0}}

## Attach lambda to eventbridge

In [122]:
lambda_client = boto3.client('lambda')
lambda_arn = 'lambda_Arn'

response_check = eventbridge_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            'Id': '1',
            'Arn': lambda_arn
        }
    ]
)

In [123]:
response

{'RuleArn': 'arn:aws:events:us-east-1:470086202700:rule/model-approval-rule-specific-group',
 'ResponseMetadata': {'RequestId': 'b67343f5-a8b9-4df0-b123-bd6ede8feb5d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b67343f5-a8b9-4df0-b123-bd6ede8feb5d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Wed, 10 Jan 2024 19:37:19 GMT'},
  'RetryAttempts': 0}}

In [124]:
# Grant EventBridge permission to invoke the Lambda function
lambda_client.add_permission(
    FunctionName='fraud-endpoint-deployment',
    StatementId='EventBridgeInvoke',
    Action='lambda:InvokeFunction',
    Principal='events.amazonaws.com',
    SourceArn=response['RuleArn']
)

{'ResponseMetadata': {'RequestId': 'd72d26e5-358d-4967-8b88-f56cf52313ae',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Wed, 10 Jan 2024 19:37:45 GMT',
   'content-type': 'application/json',
   'content-length': '378',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd72d26e5-358d-4967-8b88-f56cf52313ae'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"EventBridgeInvoke","Effect":"Allow","Principal":{"Service":"events.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:470086202700:function:fraud-endpoint-deployment","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:events:us-east-1:470086202700:rule/model-approval-rule-specific-group"}}}'}

## Approve model

In [128]:
# Update the model package status to 'Approved'
response = sagemaker_boto_client.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved"
)

print(response)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:470086202700:model-package/fraud-detection/1', 'ResponseMetadata': {'RequestId': '388f50cf-3845-4de0-a69e-fd0b81af0f8f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '388f50cf-3845-4de0-a69e-fd0b81af0f8f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Wed, 10 Jan 2024 20:21:28 GMT'}, 'RetryAttempts': 0}}
